In [1]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
from tokens import tokens
import warnings

In [2]:
# CLient info accessed here: https://developer.spotify.com/dashboard/applications/ba30303f0d2a490cb6a29af1bb3bc891
my_tokens = tokens()

CLIENT_ID = my_tokens.CLIENT_ID
CLIENT_SECRET = my_tokens.CLIENT_SECRET
username = my_tokens.username
scope_top_tracks = 'user-top-read'
scope_saved_tracks = 'user-library-read'

In [3]:
# Initiliaze an instance of the spotipy class
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope_top_tracks, client_id = CLIENT_ID, client_secret = CLIENT_SECRET, redirect_uri='http://localhost:8080', username=username))
sp2 = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope_saved_tracks, client_id = CLIENT_ID, client_secret = CLIENT_SECRET, redirect_uri='http://localhost:8080', username=username))

In [28]:
# Verify that there are more than 50 tracks (the API will only let you pull 50 at a time)
my_top_tracks = sp.current_user_top_tracks(limit = 50, offset = 50)

In [30]:
my_top_tracks['items']
# Nope, no more than 50

[]

In [ ]:
# Code to get my top two hundred tracks of the last 6 months. 
# Turns out, spotify only gives you your top 50, so this didn't work for the class assignment.

offset = 0
limit = 50

top_names = []
top_ids = []
top_release_dates = []
top_popularity = []
top_uris = []

my_top_tracks = sp.current_user_top_tracks(limit = limit, offset = offset)

for j in range(5):
    for i in range(50):
        if len(my_top_tracks['items']) >= i+1:
            try: 
                current_track = my_top_tracks['items'][i]['track']

                top_uris.append(current_track['uri'])
                top_names.append(current_track['name'])
                top_ids.append(current_track['id'])
                top_release_dates.append(current_track['album']['release_date'])
                top_popularity.append(current_track['popularity'])  
            except:
                print(i)
    
    offset = offset + 50 * j
    my_top_tracks = sp2.current_user_top_tracks(limit = limit, offset = offset)


In [4]:
# Instead, get all of my saved songs:
# This endpoint only allows you to pull 50 songs at a time, so keep going until you've pulled all your information:

offset = 0
limit = 50

names = []
ids = []
release_dates = []
popularity = []
uris = []

# I need to use the spotipy object with the library scope:
my_tracks = sp2.current_user_saved_tracks(limit = limit, offset = offset)

while(len(my_tracks['items']) > 0):
    for i in range(50):
        if len(my_tracks['items']) >= i+1:
            try: 
                current_track = my_tracks['items'][i]['track']

                uris.append(current_track['uri'])
                names.append(current_track['name'])
                ids.append(current_track['id'])
                release_dates.append(current_track['album']['release_date'])
                popularity.append(current_track['popularity'])  
            except:
                print(i)
                  
    
    offset = offset + 50
    my_tracks = sp2.current_user_saved_tracks(limit = limit, offset = offset)



In [5]:
# See how many tracks we got:
len(uris)

703

In [6]:
# Get audio features for each of these songs:

track_features = []
# You can use these to track your progress:
# i = 0
for uri in uris:
    # print(i)
    track_features.append(sp2.audio_features(uri))
    # i += 1

In [7]:
# Put those audio features into a dataframe:

audio_features = pd.DataFrame(track_features[0])

with warnings.catch_warnings(record=True):
    for i in range(1,len(track_features)):
        audio_features = audio_features.append(pd.DataFrame(track_features[i]))
    

audio_features.head()


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.529,0.664,11,-9.235,1,0.0545,0.338,0.000000,0.2950,0.653,165.894,audio_features,2sktQKiGgUdS7nnTLBtKNT,spotify:track:2sktQKiGgUdS7nnTLBtKNT,https://api.spotify.com/v1/tracks/2sktQKiGgUdS...,https://api.spotify.com/v1/audio-analysis/2skt...,185589,4
0,0.690,0.973,2,-3.588,1,0.0492,0.126,0.000000,0.1010,0.772,143.957,audio_features,1mlGScrDQqHqmhdIqE8MmA,spotify:track:1mlGScrDQqHqmhdIqE8MmA,https://api.spotify.com/v1/tracks/1mlGScrDQqHq...,https://api.spotify.com/v1/audio-analysis/1mlG...,231533,4
0,0.687,0.783,6,-4.440,1,0.0386,0.162,0.000013,0.1140,0.443,129.964,audio_features,3GCL1PydwsLodcpv0Ll1ch,spotify:track:3GCL1PydwsLodcpv0Ll1ch,https://api.spotify.com/v1/tracks/3GCL1PydwsLo...,https://api.spotify.com/v1/audio-analysis/3GCL...,231133,4
0,0.753,0.678,5,-5.421,1,0.0644,0.085,0.000002,0.1300,0.583,96.006,audio_features,1p80LdxRV74UKvL8gnD7ky,spotify:track:1p80LdxRV74UKvL8gnD7ky,https://api.spotify.com/v1/tracks/1p80LdxRV74U...,https://api.spotify.com/v1/audio-analysis/1p80...,231827,4
0,0.523,0.308,8,-10.805,1,0.0376,0.815,0.000000,0.0875,0.432,81.273,audio_features,6JrJdo0rvJZY6Yfj49ot2O,spotify:track:6JrJdo0rvJZY6Yfj49ot2O,https://api.spotify.com/v1/tracks/6JrJdo0rvJZY...,https://api.spotify.com/v1/audio-analysis/6JrJ...,166600,4


In [8]:
# Compile the basic information that we pulled above into a dataframe:

track_info = pd.DataFrame(list(zip(names, uris, popularity, release_dates)), columns = ['name', 'uri', 'popularity', 'release_date'])

In [9]:
track_info.head()

,name,uri,popularity,release_date
0,Llenar el Vacío,spotify:track:2sktQKiGgUdS7nnTLBtKNT,42,2022-05-14
1,Besos En Guerra,spotify:track:1mlGScrDQqHqmhdIqE8MmA,79,2018-10-25
2,You Belong With Me,spotify:track:3GCL1PydwsLodcpv0Ll1ch,71,2008-11-11
3,Blank Space,spotify:track:1p80LdxRV74UKvL8gnD7ky,77,2014-10-27
4,Eso Y Más,spotify:track:6JrJdo0rvJZY6Yfj49ot2O,30,2020-11-06


In [10]:
# Join the two dataframes:

my_tracks = pd.merge(track_info, audio_features, on='uri').drop(['type', 'track_href', 'analysis_url'],axis=1)
my_tracks.head()

,name,uri,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,Llenar el Vacío,spotify:track:2sktQKiGgUdS7nnTLBtKNT,42,2022-05-14,0.529,0.664,11,-9.235,1,0.0545,0.338,0.000000,0.2950,0.653,165.894,2sktQKiGgUdS7nnTLBtKNT,185589,4
1,Besos En Guerra,spotify:track:1mlGScrDQqHqmhdIqE8MmA,79,2018-10-25,0.690,0.973,2,-3.588,1,0.0492,0.126,0.000000,0.1010,0.772,143.957,1mlGScrDQqHqmhdIqE8MmA,231533,4
2,You Belong With Me,spotify:track:3GCL1PydwsLodcpv0Ll1ch,71,2008-11-11,0.687,0.783,6,-4.440,1,0.0386,0.162,0.000013,0.1140,0.443,129.964,3GCL1PydwsLodcpv0Ll1ch,231133,4
3,Blank Space,spotify:track:1p80LdxRV74UKvL8gnD7ky,77,2014-10-27,0.753,0.678,5,-5.421,1,0.0644,0.085,0.000002,0.1300,0.583,96.006,1p80LdxRV74UKvL8gnD7ky,231827,4
4,Eso Y Más,spotify:track:6JrJdo0rvJZY6Yfj49ot2O,30,2020-11-06,0.523,0.308,8,-10.805,1,0.0376,0.815,0.000000,0.0875,0.432,81.273,6JrJdo0rvJZY6Yfj49ot2O,166600,4


In [11]:
# Now, save this dataset down:

# my_tracks.to_csv('my_saved_tracks.csv')